In [1]:
import ipywidgets
import pandas as pd
import geoglows

import ipyleaflet
from io import StringIO

In [9]:
# df = pd.read_csv("E:\\NIH\\data\\IndiaWater\\reservoir_lat_long.csv")
url="https://drive.google.com/file/d/1wu05ARbjpiAnVjoHMTD3bGePKu1DTP7o/view?usp=sharing"
url2='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url2)

Basin = pd.unique(df['CWC_BASIN'])

my_map = ipyleaflet.Map(center=(23.3700354, 85.3250132), zoom=6)

# for DamName, BasName, lon, lat in df[["WRISNAME","CWC_BASIN","LONGI", "LATTI"]].values:
#     marker = ipyleaflet.Marker(location=(lat, lon), draggable=False, title=DamName+", "+BasName)
#     my_map.add_layer(marker)


# ################################################

DD1 = ipywidgets.widgets.Dropdown(options=Basin, description='Basin Name:', 
                       disabled=False, value =None, 
                       layout=ipywidgets.Layout(width='auto'),
                      style = {'description_width': 'initial'})

Dams = df.loc[df['CWC_BASIN'] == DD1.value]["WRISNAME"].unique()

DD2 = ipywidgets.widgets.Dropdown(description='Reservoirs Name:', 
                       disabled=False, value =None, 
                       layout=ipywidgets.Layout(height='auto', width='auto'),
                      style = {'description_width': 'initial'})

def updateDD2(*args):
    DD2.options = df.loc[df['CWC_BASIN'] == DD1.value]["WRISNAME"].unique()
DD1.observe(updateDD2)

#####################################################

# header = HTML("<h1>GeoGlows Forecast Application by Sukant Jain</h1>", layout=Layout(height='auto'))
# header.style.text_align='center'

style = {'description_width': 'initial'}

ForecastedStreamFlow_Button = ipywidgets.widgets.Button(
    description='Forecasted Stream Flow',
    button_style='success',
    tooltip='Click to visualize the Forecasted Stream Flow',
    layout=ipywidgets.Layout(width='260px'),
    style = {'description_width': 'initial'})
                      

FlowDuration_Button = ipywidgets.widgets.Button(
    description='Flow Duration Curve',
    button_style='danger',
    tooltip='Click to visualize the Flow Duration Curve (derived from the ERA-5 data)',
    layout=ipywidgets.Layout(width='260px'),
    style = {'description_width': 'initial'},
)

uploader = ipywidgets.widgets.FileUpload(
    accept='.csv', 
    multiple=False, 
    description="Upload Observed Data in CSV",
    layout=ipywidgets.Layout(width='260px'),
    style=style)

BiasCorrectedForecast_Button = ipywidgets.widgets.Button(
    description='Bias Corrected Forest',
    button_style='primary',
    tooltip='Click to visualize the Bias Corrected Forest',
    layout=ipywidgets.Layout(width='260px'),
    style=style)


output = ipywidgets.widgets.Output()

###############################################################

def fromLatLong(lat, long):
    
    global reach_id
    global rperiods
    global stats
    global hist
    global title
    
    
    reach_id = geoglows.streamflow.latlon_to_reach(lat, long)['reach_id']    
    rperiods = geoglows.streamflow.return_periods(reach_id)
    stats = geoglows.streamflow.forecast_stats(reach_id)
    hist = geoglows.streamflow.historic_simulation(reach_id)
    title = {'Reach ID': reach_id}

    
   
# View each of the forecasts individually
def ForecastedStreamFlow(b):
    with output:
        output.clear_output()
        
        if DD2.value == None:
            print ("First Select a Reservoir")
        else:
            print ('Displaying Forecasted Stream Flow...')    
            lat = df.loc[df['WRISNAME'] == DD2.value]["LATTI"].values[0]
            long = df.loc[df['WRISNAME'] == DD2.value]["LONGI"].values[0]

            MarkerTitle = DD2.value + ", " + DD1.value
            marker = ipyleaflet.Marker(location=(lat, long), draggable=False, title=MarkerTitle)
            my_map.add_layer(marker)
            my_map.center = (lat, long)
            
            
            fromLatLong(lat, long)

            forecast_figure = geoglows.plots.forecast_stats(stats, rperiods, titles=title)
            forecast_figure.show()
        
# Flow Duration Curve (derived from the ERA-5 data)
def FlowDuration(b):
    with output:
        output.clear_output()
        
        if DD2.value == None:
            print ("First Select a Reservoir")
        else:
            print ('Displaying Forecasted Stream Flow...')  
            lat = df.loc[df['WRISNAME'] == DD2.value]["LATTI"].values[0]
            long = df.loc[df['WRISNAME'] == DD2.value]["LONGI"].values[0]

            
            flow_duration_figure = geoglows.plots.flow_duration_curve(hist, titles=title)
            flow_duration_figure.show()

def BiasCorrection(b):
    with output:
        output.clear_output()
        
        if DD2.value == None:
            print ("First Select a Reservoir")
        else:
            print ('Displaying Bias Corrected Forecasted Stream Flow...')  
            lat = df.loc[df['WRISNAME'] == DD2.value]["LATTI"].values[0]
            long = df.loc[df['WRISNAME'] == DD2.value]["LONGI"].values[0]



            s=str(uploader.data[0],'utf-8')
            data = StringIO(s) 
            observed_data=pd.read_csv(data, index_col=0)
            observed_data.index = pd.to_datetime(observed_data.index).tz_localize('UTC')
            titles = {'Reach ID': reach_id, 'bias_corrected': True}
            
            print('Here is a preview of your data')
            print(observed_data.head(10))

            corrected_stats = geoglows.bias.correct_forecast(stats, hist, observed_data)
            geoglows.plots.forecast_stats(corrected_stats, titles=titles).show()
        
ForecastedStreamFlow_Button.on_click(ForecastedStreamFlow)
FlowDuration_Button.on_click(FlowDuration)
BiasCorrectedForecast_Button.on_click(BiasCorrection)


###############################################################

vBox1 = ipywidgets.widgets.VBox([DD1, DD2])
vBox2 = ipywidgets.widgets.VBox([ForecastedStreamFlow_Button, FlowDuration_Button])
vBox3 = ipywidgets.widgets.VBox([uploader, BiasCorrectedForecast_Button])

hBox1 =  ipywidgets.widgets.HBox([vBox2, vBox3])

vBox4 = ipywidgets.widgets.VBox([vBox1, hBox1])

# left_button = create_expanded_button('Left', 'info')
# header = widgets.VBox([DD1, DD2, ForecastedStreamFlow_Button, BiasCorrectedForecast_Button])
header = vBox4
# center_button = create_expanded_button('Center', 'warning')
# right_button = create_expanded_button('Right', 'info')
right_button = output
# footer_button = create_expanded_button('Footer', 'success')




ipywidgets.AppLayout(header=header,
          left_sidebar=None,
          center=my_map,
          right_sidebar=right_button,
          footer=None,
         pane_widths=[0,3, 4],)

AppLayout(children=(VBox(children=(VBox(children=(Dropdown(description='Basin Name:', layout=Layout(width='aut…